In [ ]:
from modules.load_data.get_financial import get_data, generate_data

try:
    data = get_data(0)
    # print(data)
except Exception as e:
    data = generate_data(start_date, end_date, 0)

# print(data)
x, y = data, data[:, -1].astype(np.float32)
scaler = y[:int(len(x) * config.density)].astype(np.float32)
y = (y - np.mean(scaler)) / np.std(scaler)
# print(scaler.shape)
x[:, -1] = x[:, -1].astype(np.float32)
temp = x[:, -1].astype(np.float32)
x[:, -1] = (temp - np.mean(scaler)) / np.std(scaler)
x = x.astype(np.float32)
y = y.astype(np.float32)
X_window, y_window = create_window_dataset(x, y, config.seq_len, config.pred_len)

